# PRÁCTICA 1 - APRENDIZAJE SUPERVISADO - SUPERVISED

## 0. Indice
1. [Importación de Librerías, Lectura y Configuración](#21-importación-de-librerías-lectura-y-configuración)
2. [Metodos Clásicos](#22-metodos-classicos)
    1. [Regresión Lineal](#221-regresion-lineal)
    2. [Regresión Polinomial](#222-regresion-polinomial)
    3. [Regresión Logística](#223-regresion-logistica)



## 2.1. Importación de Librerías, Lectura y Configuración

Es este apartado incluiremos las configuraciones iniciales pertinentes para la realizacion del aprendizaje supervisado.

### 2.1.1. Imports

Procederemos ha realizar los imports pertinentes

In [9]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, f1_score, make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

### 2.1.2. Lectura de Dataframes

En este apartado procederemos a la lectura del dataframe creado en el notebook de preprocesado.
Esto constara del csv con la informacion de los clientes "X" y el csv con las labels "y".

In [10]:
X_train = pd.read_csv("./data/output/clean_train_data.csv")
X_test = pd.read_csv("./data/output/clean_test_data.csv")
y_train = pd.read_csv("./data/output/clean_train_label.csv")
y_test = pd.read_csv("./data/output/clean_test_label.csv")

## 2.2. Metodos Classicos

Se realizaran pruebas con los metodos:
1. **Regresión Lineal**
2. **Regresión Polinomial**
3. **Regresión Lógica**

### 2.2.1. Regresion Lineal

- Para realizar la regresion lineal procederemos a utilizar el algoritmo de `LinealRegression`.
- Para poder testear el modelo utilizaremos los métodos de `MSE` y `R2`

In [11]:
lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

y_train_pred = lin_reg.predict(X_train)
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)
print(f'Error cuadrático medio (MSE) del entrenamiento: {mse_train}')
print(f'Coeficiente de determinacion (R2) del entrenamiento: {r2_train}')

y_test_pred = lin_reg.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
print(f'Error cuadrático medio (MSE) del testing: {mse_test}')
print(f'Coeficiente de determinacion (R2) del testing: {r2_test}')

Error cuadrático medio (MSE) del entrenamiento: 0.24945396256829724
Coeficiente de determinacion (R2) del entrenamiento: 0.0021841497268110244
Error cuadrático medio (MSE) del testing: 0.24949968198253755
Coeficiente de determinacion (R2) del testing: 0.002001272069849791


- Como podemos ver con los valores de MSE y R2, **no hay una realacion lineal fuerte** entre las variables por lo que podemos deducir que no hay una alta relacion lineal entre las variables predictorias y la variable objetivo.
- También podemos ver que la diferencia entre training y testing soy muy pequeñas por lo que parece que **no hay overfitting**.

### 2.2.2. Regresion Polinomial

- Para realizar la regresion polinomial procederemos a utilizar el algoritmo de `PolinomialFeatures` para añadir el cuadrado del valor de las caracteristicas y posteriormente entrenarlo con una `LinealRegression`.
- Para poder testear el modelo utilizaremos los métodos de `MSE` y `R2`

In [12]:

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.fit_transform(X_test)

lin_reg = LinearRegression()

lin_reg.fit(X_train_poly, y_train)

y_train_pred = lin_reg.predict(X_train_poly)
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)
print(f'Error cuadrático medio (MSE) del entrenamiento: {mse_train}')
print(f'Coeficiente de determinacion (R2) del entrenamiento: {r2_train}')

y_test_pred = lin_reg.predict(X_test_poly)
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
print(f'Error cuadrático medio (MSE) del testing: {mse_test}')
print(f'Coeficiente de determinacion (R2) del testing: {r2_test}')


Error cuadrático medio (MSE) del entrenamiento: 0.2431847924180999
Coeficiente de determinacion (R2) del entrenamiento: 0.027260830327600405
Error cuadrático medio (MSE) del testing: 0.24872565426137788
Coeficiente de determinacion (R2) del testing: 0.005097382954488472


- Como podemos volver a ver, el modelo polinomial **sigues sin campturar adecuadamente la relacion entre las caracteristicas y la variable objetivo**.
- También podemos ver que la diferencia entre training y testing soy muy pequeñas por lo que parece que **no hay overfitting**.

### 2.2.3. Regresion Logistica

- Para realizar la regresion logistica procederemos a utilziar el algoritmo de `LogisticalRegression` de sklearn.
- Para poder testear el modelo utilizaremos el método de `f1_score`.

In [13]:

model = LogisticRegression(max_iter=1000)
params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['saga', 'liblinear']
}
f1_scorer = make_scorer(f1_score)

grid_search = GridSearchCV(model, params, scoring=f1_scorer, cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor F1-score en validación:", grid_search.best_score_)

C:\Users\peric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
50 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\peric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\peric\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-pac

Mejores hiperparámetros: {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}
Mejor F1-score en validación: 0.5461678455913518


- Como podemos observar, usando una regresion logistica **tampoco consigue valores esperanzadores** aun intentando seleccionar los mejores hiperparametros por lo que probaremos con algoritmos mas complejos.

## 2.3. 